# Data Augmentation
## Purpose of this notebook is to take both prepped datasets and combined into a singular dataset ready to train models with

In [251]:
# Import required libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import datetime
import os.path

In [252]:
# Constants for changing
WINDOW_SIZE = 20
SENSORS = ['back_angle', 'left_angle', 'right_angle']# ,'back_1der' , 'left_1der', 'right_1der', 'back_2der', 'left_2der', 'right_2der']
PLOT_ON = False

### First retrieve the patient data

In [253]:
# Retrieve data and place with pd dataframe
patient_file_path = '/Users/jamesmeyer/University of Bath/Patient Simulator FYP - General/datasets/patient/'
patient_file_name = 'patient_data.csv'
dataframe = pd.read_csv(f'{patient_file_path}{patient_file_name}', parse_dates=['datetime'])
print(dataframe.head())

   action  subject  frame  index  back_angle  left_angle  right_angle  \
0     NaN      NaN    NaN    0.0    0.424307    0.572584     0.215395   
1     NaN      NaN    NaN    1.0    0.456290    0.568733     0.200416   
2     NaN      NaN    NaN    2.0    0.488273    0.564883     0.200416   
3     NaN      NaN    NaN    3.0    0.530917    0.557181     0.201110   
4     NaN      NaN    NaN    4.0    0.573561    0.541779     0.200416   

                 datetime  back_1der  left_1der  right_1der     back_2der  \
0 2011-12-01 11:04:05.000   0.028175   0.000385   -0.011288  4.568992e-03   
1 2011-12-01 11:04:05.050   0.032744  -0.003466   -0.007107  4.568992e-03   
2 2011-12-01 11:04:05.100   0.037313  -0.007316   -0.002926  4.568992e-03   
3 2011-12-01 11:04:05.150   0.038380  -0.013092    0.000083 -2.775558e-16   
4 2011-12-01 11:04:05.200   0.037313  -0.018868   -0.000083 -4.568992e-03   

   left_2der  right_2der  
0  -0.003851    0.004181  
1  -0.003851    0.004181  
2  -0.003851    0

In [254]:
# Obtaining only the required columns
patient_df = dataframe.loc[:,SENSORS]
patient_df.loc[:,'Date'] = pd.to_datetime(dataframe.loc[:,'datetime'])
patient_df.tail()

,back_angle,left_angle,right_angle,Date
38182,0.626866,0.276088,0.527046,2011-12-01 11:35:54.100
38183,0.626866,0.276088,0.525659,2011-12-01 11:35:54.150
38184,0.616205,0.276088,0.524272,2011-12-01 11:35:54.200
38185,0.616205,0.276088,0.524272,2011-12-01 11:35:54.250
38186,0.616205,0.276088,0.524272,2011-12-01 11:35:54.300


In [255]:
# Initial plot of data
if PLOT_ON:
    patient_df.plot(subplots=True, figsize=(16, 16), x='Date')

In [256]:
# Split test and train data
# patient_train = patient_df.loc[patient_df['Date'] <= '2011-12-01 11:38:00'] 
# patient_test = patient_df.loc[patient_df['Date'] > '2011-12-01 11:38:00']
# print("Not really being used at this stage")

# Split train and test data according to the new division 29/04/21
major = int(patient_df.shape[0] * 0.8)
major_train = patient_df.iloc[:major, :]
major_test = patient_df.iloc[major:, :]

print(major_test.shape)

minor = int(patient_df.shape[0] * 0.4)
minor_train = patient_df.iloc[:minor, :]
minor_test = patient_df.iloc[minor:, :]

print(minor_test.shape)

full_test = patient_df.iloc[:, :]

print(full_test.shape)

(7638, 4)
(22913, 4)
(38187, 4)


### Then retrieve the control data

In [257]:
# Location of processed control data
control_file_path = '/Users/jamesmeyer/University of Bath/Patient Simulator FYP - General/datasets/control/'
control_file_name = 'control_data.csv'

# Load the data in pd df
control_df = pd.read_csv(f'{control_file_path}{control_file_name}', index_col=False)
control_df.head()

,action,subject,frame,back_angle,left_angle,right_angle,back_1der,left_1der,right_1der,back_2der,left_2der,right_2der
0,1,1,1,0.309028,0.052406,0.991990,-0.059143,-0.018081,0.000857,0.024875,0.006771,-0.000014
1,1,1,2,0.212165,0.023802,0.993044,-0.034267,-0.011310,0.000843,0.024875,0.006771,-0.000014
2,1,1,3,0.251407,0.033033,0.993763,-0.009392,-0.004539,0.000830,0.024875,0.006771,-0.000014
3,1,1,4,0.232627,0.025384,0.994412,0.006291,-0.000403,0.000699,-0.008194,-0.002490,-0.000081
4,1,1,5,0.251839,0.028918,0.995454,0.002175,-0.001812,0.000567,0.005269,0.001213,-0.000224


### Now to convert both to windowed sequences and combine for model training

In [258]:
def to_sequences(x, seq_size=1):
    '''Function to return windowed versions'''
    x_values = []

    for i in range(len(x)-seq_size):
        x_values.append(x.iloc[i:(i+seq_size)].values)
        
    return np.array(x_values)

In [259]:
# Convert all to sequences

# patient_major_train
# patient_major_test
# patient_minor_train 
# patient_minor_test
# patient_full_test

patient_major_train = to_sequences(major_train[SENSORS], WINDOW_SIZE)
patient_major_test = to_sequences(major_test[SENSORS], WINDOW_SIZE)
patient_minor_train = to_sequences(minor_train[SENSORS], WINDOW_SIZE)
patient_minor_test = to_sequences(minor_test[SENSORS], WINDOW_SIZE)
patient_full_test = to_sequences(full_test[SENSORS], WINDOW_SIZE)

In [260]:
# Control data is harder because the data itself is segmented between individuals and actions so only windows from same action and person can be extracted

# Loop through each action and subject
action_nos = [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 14]
subject_nos = list(range(1,11))

extract_data = []
for action in action_nos:
    for subject in subject_nos:
        sample = control_df.loc[(control_df['action'] == action) & (control_df['subject'] == subject), SENSORS]
        control_samples = to_sequences(sample, WINDOW_SIZE)
        extract_data.extend(control_samples)
control_train = np.array(extract_data)

In [261]:
print(f'Number of control training samples: {control_train.shape}')

Number of control training samples: (12022, 20, 3)


## Combinations

In [262]:
augment_train = np.append(patient_minor_train, control_train, axis=0)
print(f'Number of combined training samples: {augment_train.shape}')

Number of combined training samples: (27276, 20, 3)


In [263]:
# Set up saves
output_file_path = '/Users/jamesmeyer/University of Bath/Patient Simulator FYP - General/datasets/'

deriv_flag = False
for data in SENSORS:
    if 'der' in data:

        deriv_flag = True
        break

output_id = f'ws{WINDOW_SIZE}num{len(SENSORS)}der{deriv_flag}'

In [264]:
path = output_file_path + output_id

np.save(f'{path}-patient-train.npy', patient_major_train)
np.save(f'{path}-patient-major-test.npy', patient_major_test)
np.save(f'{path}-augment-train.npy', augment_train)
np.save(f'{path}-patient-minor-test.npy', patient_minor_test)
np.save(f'{path}-control-train.npy', control_train)
np.save(f'{path}-patient-full-test.npy', patient_full_test)


print(f'Saved to: {path}')

Saved to: /Users/jamesmeyer/University of Bath/Patient Simulator FYP - General/datasets/ws20num3derFalse


In [265]:
major_test.to_csv (f'{output_file_path}major-test-{WINDOW_SIZE}.csv', index=False)
minor_test.to_csv (f'{output_file_path}minor-test-{WINDOW_SIZE}.csv', index=False)
full_test.to_csv (f'{output_file_path}full-test-{WINDOW_SIZE}.csv', index=False)